# Setting up the environment

In [ ]:
# Import and/or install libraries

import subprocess, os

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee

try:
    import rio_tiler.io
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'rio-tiler'])
    import rio_tiler.io

In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_PATH_TO_KEY'))
ee.Initialize(credentials)

# Getting started

We will start by using Google Dynamic World and using the built layer. It seems like we should also filter the certainty to be greater than 0.06 (highest false positive in the grassland had a certainty of 0.0569).

In [ ]:
# AOI

aoi = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              28.06574851376746,
              -22.324553464205877
            ],
            [
              28.06574851376746,
              -26.01523517180692
            ],
            [
              33.733773120836446,
              -26.01523517180692
            ],
            [
              33.733773120836446,
              -22.324553464205877
            ],
            [
              28.06574851376746,
              -22.324553464205877
            ]
          ]
        ],
      })

# Prepare layers

First, we experiment with 2022 for our AOI.

In [ ]:
year = 2022
start_date = '{}-01-01'.format(year)
end_date = '{}-01-01'.format(year + 1)

# Google Dynamic World
people = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('label').eq(6).selfMask()
certainty_mask = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built').gt(0.06).selfMask()
best_people = people.mask(certainty_mask).eq(1).selfMask()
# people_c = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built').mask(people)

Rasterise Google's Open Buildings

In [ ]:
# Google Open Buildings
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v2/polygons').filter('confidence >= 0.70');

buildings_raster = buildings.reduceToImage(
  properties=['confidence'],
  reducer=ee.Reducer.median()
).gt(0).selfMask().select(['median'], ['label'])

In [ ]:
# Join with other layers
built = best_people.unmask(0).add(buildings_raster.unmask(0)).gt(0).selfMask()


# Export layers

In [ ]:
# Set configurations for the export
exportConfig = {
    'image': built, 
    'description': 'built2022',
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': 'COGS/built/built2022/built2022',
    'scale': 30,
    'maxPixels': 290234370,
    'region': aoi,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
}

task = ee.batch.Export.image.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

# Map

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')

# Map.addLayer(people, {'min':0, 'max':1, 'palette':['white','blue']}, 'people')
# Map.addLayer(best_people, {'min':0, 'max':1, 'palette':['white','red']}, 'best_people')

# Map.addLayer(buildings, {'color': 'red'}, 'Buildings confidence >= 0.70');
# Map.addLayer(buildings_raster, {'min':0, 'max':1, 'palette':['white','red']}, 'buildings_raster')
# Map.addLayer(gdw, {'min':0, 'max':8, 'palette':['419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b', 'a59b8f', 'b39fe1']}, 'GDW')
# Map.addLayer(people_c, {'min':0, 'max':0.1, 'palette':['white','blue']}, 'people_c')

Map.addLayer(built, {}, 'built')

Map.setCenter(31.944, -25.087, 16)
Map
